In [36]:
import random
import time

class QuickSort:
    def __init__(self):
        self.arr = None
        self.random_seed = None
        
    def set_random_seed(self, seed):
        self.random_seed = seed
        
    def fit(self, arr):
        self.arr = arr
        
    def partition(self, low, high, pivot_index):
        # swap the pivot with the last element
        self.arr[pivot_index], self.arr[high] = self.arr[high], self.arr[pivot_index]
        
        pivot = self.arr[high]
        i = low - 1
        for j in range(low, high):
            if self.arr[j] <= pivot:
                i += 1
                self.arr[i], self.arr[j] = self.arr[j], self.arr[i]
                
        self.arr[i + 1], self.arr[high] = self.arr[high], self.arr[i + 1]
        
        return i + 1
    
    def find_median_of_medians(self, arr):
        if len(arr) <= 5:
            return sorted(arr)[len(arr) // 2]
        
        # Divide the array into subarrays of size 5 or less
        subarrays = [arr[j : j + 5] for j in range(0, len(arr), 5)]
        
        # Calculate the median of each subarray
        medians = [sorted(subarray)[len(subarray) // 2] for subarray in subarrays]
        
        median_of_medians = self.find_median_of_medians(medians)
        
        return median_of_medians
    
    def select_pivot_median_of_medians(self, low, high):
        '''return the index of median_of_medians'''
        pivot = self.find_median_of_medians(self.arr[low : high + 1])
        return self.arr.index(pivot)
    
    def select(self, low, high, i):
        ''' return the value of the ith order statistic i = 0..(n-1)'''
        if (high - low + 1) == 1:
            return self.arr[low]
        
        pivot = self.find_median_of_medians(self.arr[low : high + 1])
        pivot_index = self.arr.index(pivot)
        
        pivot_idx_after_part = self.partition(low, high, pivot_index)
        
        # print("pivot: {}, pivot_index: {}".format(pivot, pivot_index))
        # print("partitioned array: ", self.arr[low : high + 1])
        
        if i == pivot_idx_after_part:
            return self.arr[pivot_idx_after_part]
        elif i < pivot_idx_after_part:
            return self.select(low, pivot_idx_after_part - 1, i)
        else:
            return self.select(pivot_idx_after_part + 1, high, i - pivot_idx_after_part - 1)

    def select_pivot_exact_median(self, low, high):
        pivot = self.select(low, high, (low + high + 1) // 2)
        # print("low: {}, high: {}, pivot_index: {}, pivot: {}".format(low, high, self.arr.index(pivot), pivot))
        return self.arr.index(pivot)
    
    def select_pivot_random(self, low, high):
        self.set_random_seed(10)
        random.seed(self.random_seed)
        return random.randint(low, high)
    
    def select_pivot_first(self, low, high):
        # print("pivot: ", self.arr[low])
        return low
    
    def select_pivot_last(self, low, high):
        # print("pivot: ", self.arr[high])
        return high
    
    def select_pivot_median_of_three(self, low, high):
        mid = (low + high) // 2
        candidates = [(low, self.arr[low]), (mid, self.arr[mid]), (high, self.arr[high])]
        median = sorted(candidates, key=lambda x: x[1])[1]
        # print("pivot: ", median[1])
        return median[0]
    
    def select_pivot_method(self, method):
        if method == "random":
            return self.select_pivot_random
        elif method == "median_of_medians":
            return self.select_pivot_median_of_medians
        elif method == "exact_median":
            return self.select_pivot_exact_median
        elif method == "first":
            return self.select_pivot_first
        elif method == "last":
            return self.select_pivot_last
        elif method == "median_of_three":
            return self.select_pivot_median_of_three
        else:
            raise ValueError("Invalid pivot selection method")
    
    def quicksort(self, low, high, select_method="random"):
        select_pivot = self.select_pivot_method(select_method)
        
        if low < high:
            pivot_index = select_pivot(low, high)
            pivot_idx_after_part = self.partition(low, high, pivot_index)
            self.quicksort(low, pivot_idx_after_part - 1, select_method)
            self.quicksort(pivot_idx_after_part + 1, high, select_method)
            
    def print_array(self):
        print(self.arr)

In [12]:
sorter = QuickSort() # Instantiate the QuickSort class
arr = generate_random_array(20)
sorter.fit(arr)
sorter.print_array()
sorter.quicksort(0, 20 - 1, select_method="random")
sorter.print_array()

[2, 9, 3, 14, 19, 20, 17, 1, 7, 16, 11, 6, 10, 12, 13, 15, 5, 4, 18, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


# Experiments

In [37]:
import platform

# Get system-related information
system = platform.system()
processor = platform.processor()
python_version = platform.python_version()

# Display the experiment environment
print("Experiment Environment:")
print(f"Operating System: {system}")
print(f"Processor: {processor}")
print(f"Python Version: {python_version}")


Experiment Environment:
Operating System: Darwin
Processor: arm
Python Version: 3.9.12


In [45]:
def generate_random_array(n):
    random.seed(123)
    arr = random.sample(range(1, n+1), n)
    return arr

# Generate an array of a given size in ascending order
def generate_sorted_array(n):
    return list(range(n))

# Generate an array of a given size in descending order
def generate_reverse_sorted_array(size):
    return list(range(size, 0, -1))

def measure_quicksort_execution_time(array_generator, array_size, num_iterations, pivot_selection_method):
    execution_times = []
    sorter = QuickSort() # Instantiate the QuickSort class

    execution_times = []
    for _ in range(num_iterations):
        arr = array_generator(array_size)
        # Set the array to be sorted using the fit function
        sorter.fit(arr)
        
        start_time = time.time()
        sorter.quicksort(0, array_size - 1, pivot_selection_method)
        end_time = time.time()
        execution_time = end_time - start_time
        execution_times.append(execution_time)
    
    return execution_times

def run_experiment(array_generator, array_size, num_iterations):
    execution_times = {
        "Median of Medians": None, 
        "Median of Three": None, 
        "Exact Medians": None, 
        "Random": None, 
        "Last": None
    }
    
    # Measure the execution times for each pivot selection
    execution_times["Median of Medians"] = measure_quicksort_execution_time(array_generator, array_size, num_iterations, "median_of_medians")
    execution_times["Median of Three"] = measure_quicksort_execution_time(array_generator, array_size, num_iterations, "median_of_three")
    execution_times["Exact Medians"] = measure_quicksort_execution_time(array_generator, array_size, num_iterations, "exact_median")
    execution_times["Random"] = measure_quicksort_execution_time(array_generator, array_size, num_iterations, "random")
    try:
        execution_times["Last"] = measure_quicksort_execution_time(array_generator, array_size, num_iterations, "last")
    except RecursionError:
        print("Recursion depth exceeded for pivot selection method: Last")
    
    print("array size: {}, num_iterations: {}".format(array_size, num_iterations))
    print()
    for technique, times in execution_times.items():
        try:
            avg_execution_time = round(sum(times) / len(times), 5)
            print(f"{technique}: {avg_execution_time} seconds")
        except:
            print(f"{technique}: recursion depth exceeded")
        
    print("------------------------------------------------------------------")

In [39]:
num_iterations = 10
array_sizes = [1000, 10000, 50000]
print("unsorted array: ")
print("------------------------------------------------------------------")
for array_size in array_sizes:
    run_experiment(generate_random_array, array_size, num_iterations)

unsorted array: 
------------------------------------------------------------------
array size: 1000, num_iterations: 10

Median of Medians: 0.0079 seconds
Median of Three: 0.00182 seconds
Exact Medians: 0.01194 seconds
Random: 0.0066 seconds
Last: 0.00172 seconds
------------------------------------------------------------------
array size: 10000, num_iterations: 10

Median of Medians: 0.21473 seconds
Median of Three: 0.02217 seconds
Exact Medians: 0.64904 seconds
Random: 0.07108 seconds
Last: 0.02122 seconds
------------------------------------------------------------------
array size: 50000, num_iterations: 10

Median of Medians: 4.67672 seconds
Median of Three: 0.12962 seconds
Exact Medians: 14.28389 seconds
Random: 0.38024 seconds
Last: 0.13583 seconds
------------------------------------------------------------------


In [48]:
num_iterations = 10
array_sizes = [1000, 10000, 50000]
print("sorted array: ")
print("last element pivot selection is ignored since the partition process exceeds the maximum recursion depth when n = 3000")
print("------------------------------------------------------------------")
for array_size in array_sizes:
    run_experiment(generate_sorted_array, array_size, num_iterations)

sorted array: 
last element pivot selection is ignored since the partition process exceeds the maximum recursion depth when n = 3000
------------------------------------------------------------------
array size: 1000, num_iterations: 10

Median of Medians: 0.01396 seconds
Median of Three: 0.0015 seconds
Exact Medians: 0.02282 seconds
Random: 0.00724 seconds
Last: 0.07887 seconds
------------------------------------------------------------------
Recursion depth exceeded for pivot selection method: Last
array size: 10000, num_iterations: 10

Median of Medians: 0.31855 seconds
Median of Three: 0.01907 seconds
Exact Medians: 0.95826 seconds
Random: 0.07873 seconds
Last: recursion depth exceeded
------------------------------------------------------------------
Recursion depth exceeded for pivot selection method: Last
array size: 50000, num_iterations: 10

Median of Medians: 6.352 seconds
Median of Three: 0.11567 seconds
Exact Medians: 19.23203 seconds
Random: 0.40909 seconds
Last: recursio

In [50]:
num_iterations = 10
array_sizes = [1000, 10000, 50000]
print("reverse sorted array: ")
print("last element pivot selection is ignored since the partition process exceeds the maximum recursion depth when n = 3000")
print("------------------------------------------------------------------")
for array_size in array_sizes:
    run_experiment(generate_reverse_sorted_array, array_size, num_iterations)

reverse sorted array: 
last element pivot selection is ignored since the partition process exceeds the maximum recursion depth when n = 3000
------------------------------------------------------------------
array size: 1000, num_iterations: 10

Median of Medians: 0.0082 seconds
Median of Three: 0.00249 seconds
Exact Medians: 0.01311 seconds
Random: 0.00665 seconds
Last: 0.05155 seconds
------------------------------------------------------------------
Recursion depth exceeded for pivot selection method: Last
array size: 10000, num_iterations: 10

Median of Medians: 0.21925 seconds
Median of Three: 0.03358 seconds
Exact Medians: 0.653 seconds
Random: 0.07327 seconds
Last: recursion depth exceeded
------------------------------------------------------------------
Recursion depth exceeded for pivot selection method: Last
array size: 50000, num_iterations: 10

Median of Medians: 4.80904 seconds
Median of Three: 0.20177 seconds
Exact Medians: 14.44561 seconds
Random: 0.38254 seconds
Last: 